In [ ]:
from utils.metrics_ingestion import *

"""
Directory containing metrics with a following structure

parsed-logs/
├─ aws/
│  ├─ t3.large/
│  │  ├─ montage__2.0__1.0.0__2020-06-01-19-02-29-05-13-17-08-51/
│  │  │  ├─ job_descriptions.jsonl
│  │  │  ├─ metrics.jsonl
│  │  │  ├─ sys_info.jsonl
│  │  ├─ soykb__134__1.0.0__2020-06-21-01-30-50-05-13-17-08-51/
│  │  │  ├─ ...
├─ gcloud/
│  ├─ n1-standard-4/
│  │  ├─ montage2__1.0__1.0.0__2020-06-21-19-34-03/
│  │  │  ├─ ...
│  ├─ n2-standard-4/
│  │  ├─ soykb__446__1.0.0__2020-06-23-08-02-29/
│  │  │  ├─ ...
"""
base_path = Path("parsed-logs")

In [ ]:
"""
Prepare providers data paths
"""
providers = logs(base_path)

In [ ]:
"""
Parse the metrics located underneath the paths of each provider
"""
hf_data = parse_metrics_data(providers)

In [ ]:
"""
Extract the labels used to convert the logs from workflow execution into OpenMetric standard
"""
hf_labels = parse_metrics_labels(providers)

In [ ]:
"""
Generates OpenMetrics for each of the workflow executions,
and save the in each of the logs folder under the name `metrics.om`
"""
generate_open_metrics(providers, hf_labels)

In [ ]:
%%bash
# create prometheus blocks from metrics.om files
for m in parsed-logs/**/metrics.om;
    prlimit --nofile=8192 promtool tsdb create-blocks-from openmetrics $m prometheus-db;
end

In [ ]:
# start the prometheus server
!docker run -d -p 9090:9090 \
    -v prometheus-db:/prometheus \
    --name prometheus \
    prom/prometheus \
    --storage.tsdb.retention.time=3y \
    --config.file=/etc/prometheus/prometheus.yml \
    --storage.tsdb.path=/prometheus \
    --storage.tsdb.allow-overlapping-blocks \
    --query.lookback-delta=3s \
    --web.console.libraries=/usr/share/prometheus/console_libraries \
    --web.console.templates=/usr/share/prometheus/consoles